In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/threads-an-instagram-app-reviews/threads_reviews.csv


## Load Data

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('/kaggle/input/threads-an-instagram-app-reviews/threads_reviews.csv')

In [3]:
data.head()

,source,review_description,rating,review_date
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43


In [4]:
df = data.loc[:, ['review_description', 'rating']]
df.dropna(inplace=True)

In [5]:

df = df[df.loc[:, ('rating')] != 3]
df['postive'] = np.where(df.loc[:,('rating')]>3, 1, 0)
df = df.rename(columns={"review_description": "review"})
df.head()

,review,rating,postive
0,Meh. Not the greatest experience on a Chromebo...,2,0
5,"Currently, it's very challenging to use. It's ...",2,0
6,I still don’t want to see content from people ...,1,0
7,Could be great if all pages loaded when you cl...,2,0
9,Not bad! This is its first launch. There is st...,5,1


In [6]:
df['postive'].mean()

0.6200494641384996

In [21]:
X = df['review']
y = df['postive']
# train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=42)

In [22]:
print(X_train.shape, y_train.shape)
print(X_train[0])

(21227,) (21227,)
Meh. Not the greatest experience on a Chromebook. Seems to be customized for phones only. Opens in a little screen that you can't expand or resize - for reasons that are a complete mystery to me. Judging from the fact that every other app I know of is resizeable, this seems like it was a conscious choice by the developers . Why you'd do something like this is beyond understanding and suggests a control freak approach. Not a great way to make a first impression.


## CountVectorizer

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3).fit(X_train)
feature_names = vectorizer.get_feature_names_out()

In [31]:
len(feature_names)

14973

In [32]:
X_train_vectorized = vectorizer.transform(X_train)
print(X_train_vectorized.shape, y_train.shape)

(21227, 14973) (21227,)


### Modeling

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

LogisticRegression(max_iter=1000)

In [35]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vectorizer.transform(X_test))
print("[ METRICS ]", roc_auc_score(y_test, predictions))

[ METRICS ] 0.8524273872628703


In [36]:
# additional
model.coef_

array([[ 0.12444023,  0.50004152,  0.68562996, ..., -1.09245899,
        -0.315108  ,  0.4217891 ]])